In [1]:
# Importando todos las Librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

# Modelos a experimentar
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Librerías para MLflow (importante para el seguimiento de experimentos)
import mlflow
import mlflow.sklearn
import logging
import yaml # Para leer params.yaml 
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

# Configurar MLflow (opcional, si quieres un server remoto o un archivo específico)
mlflow.set_tracking_uri("http://127.0.0.1:5000/") # Si tienes un servidor MLflow local
mlflow.set_experiment("Absenteeism_Prediction_Experiments")

2025/10/12 11:30:00 INFO mlflow.tracking.fluent: Experiment with name 'Absenteeism_Prediction_Experiments' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/383079581240596013', creation_time=1760290200678, experiment_id='383079581240596013', last_update_time=1760290200678, lifecycle_stage='active', name='Absenteeism_Prediction_Experiments', tags={}>

In [2]:
# Ruta del dataset procesad
ruta_dataset = '../data/interim/absenteeism_eda_fe_intermediate.csv'

try:
    df = pd.read_csv(ruta_dataset)
    print(f"✅ Dataset cargado correctamente desde: {ruta_dataset}")
    print(f"Dimensiones del dataset: {df.shape}")
    print("\nPrimeras 5 filas del dataset:")
    display(df.head()) # Usa display() en Jupyter para una mejor visualización
except Exception as e:
    logger.exception(f"No se pudo cargar el dataset. Asegúrate de que la ruta sea correcta y el archivo exista. Error: {e}")

✅ Dataset cargado correctamente desde: ../data/interim/absenteeism_eda_fe_intermediate.csv
Dimensiones del dataset: (514, 79)

Primeras 5 filas del dataset:


,ID,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,Hit target,Disciplinary failure,Son,Social drinker,...,Education_High school,Education_Postgraduate,BMI_category_Obese,BMI_category_Overweight,Age_group_Middle-aged,Age_group_Senior,Service_group_Medium,Service_group_Long,Distance_group_Moderate,Distance_group_Far
0,-0.589167,1.298575,0.372291,0.043166,-0.550156,-0.81621,0.664538,-0.195918,0.819848,0.969341,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,-1.307203,-0.673158,1.435389,1.502873,0.427646,-0.81621,0.664538,-0.195918,-0.865760,0.969341,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2,-0.948185,1.119326,-1.824778,0.335108,0.623206,-0.81621,0.664538,-0.195918,0.819848,0.969341,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,-0.589167,1.298575,0.372291,0.043166,-0.550156,-0.81621,0.664538,-0.195918,0.819848,0.969341,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4,-1.307203,-0.673158,1.435389,1.502873,0.427646,-0.81621,0.664538,-0.195918,-0.865760,0.969341,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0


## Preparación para las variables

In [3]:
# --- Definir la Variable Objetivo (Target) ---
# La columna 'Absenteeism time in hours' es nuestro punto de partida.
# Vamos a crear un problema de clasificación binaria para predecir si una ausencia será larga o corta.
# Usaremos la mediana como umbral para hacer esta división.

# Calcular la mediana de las horas de ausentismo
try:
    median_hours = df['Absenteeism time in hours'].median()
    print(f"La mediana de las horas de ausentismo es: {median_hours:.2f} horas.")
    print("Crearemos una variable objetivo binaria usando este umbral.")

    # Crear la variable objetivo: 1 si es mayor que la mediana, 0 en caso contrario.
    df['Target_Binary'] = (df['Absenteeism time in hours'] > median_hours).astype(int)
    print("\nDistribución de la nueva variable objetivo 'Target_Binary':")
    print(df['Target_Binary'].value_counts(normalize=True))

except KeyError:
    print("Error: Asegúrate de que la columna 'Absenteeism time in hours' exista en tu DataFrame.")
    # Detener la ejecución si la columna no existe.
    # Y luego re-intentar las operaciones.


# --- Definir Características (X) y Variable Objetivo (y) ---
# Ahora separamos el DataFrame en nuestras variables predictoras (X) y la variable que queremos predecir (y).

# 'y' es la columna binaria que acabamos de crear.
y = df['Target_Binary']

# 'X' son todas las demás columnas, EXCEPTO:
#  - 'ID': Es un identificador, no un predictor.
#  - 'Absenteeism time in hours': La usamos para crear el target, incluirla sería hacer trampa (data leakage).
#  - 'Target_Binary': Es nuestra variable objetivo.
X = df.drop(columns=['ID', 'Absenteeism time in hours', 'Target_Binary'])

print("\nForma de las características (X):", X.shape)
print("Forma de la variable objetivo (y):", y.shape)
print("\nSe usarán las siguientes columnas como características (predictores):")
print(X.columns.tolist())


# ---División en Conjuntos de Entrenamiento y Prueba ---
# Dividimos los datos para poder entrenar el modelo con una parte y evaluarlo con datos que nunca ha visto.
# - test_size=0.2: Usamos el 20% de los datos para la prueba.
# - random_state=42: Asegura que la división sea siempre la misma para que los resultados sean reproducibles.
# - stratify=y: ¡Muy importante! Asegura que la proporción de ausencias largas y cortas sea la misma en
#   el conjunto de entrenamiento y en el de prueba.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("\nForma de los conjuntos de datos divididos:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")


# ---Escalado de Características ---
# Muchos modelos funcionan mejor cuando las características numéricas tienen una escala similar.
# StandardScaler las transforma para que tengan una media de 0 y una desviación estándar de 1.
# OJO: El escalador se 'ajusta' (fit) SOLO con los datos de entrenamiento para evitar fuga de información.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✅ Preprocesamiento y división de datos completados. Los datos están listos para el entrenamiento.")

La mediana de las horas de ausentismo es: 3.00 horas.
Crearemos una variable objetivo binaria usando este umbral.

Distribución de la nueva variable objetivo 'Target_Binary':
Target_Binary
0    0.595331
1    0.404669
Name: proportion, dtype: float64

Forma de las características (X): (514, 77)
Forma de la variable objetivo (y): (514,)

Se usarán las siguientes columnas como características (predictores):
['Transportation expense', 'Distance from Residence to Work', 'Service time', 'Age', 'Work load Average/day', 'Hit target', 'Disciplinary failure', 'Son', 'Social drinker', 'Social smoker', 'Pet', 'Weight', 'Height', 'Body mass index', 'Lifestyle_risk_score', 'Dependents_count', 'Has_dependents', 'Has_family_or_pets', 'Healthy_lifestyle', 'Penalty_risk_score', 'Reliability_score', 'Reliability_score_norm', 'Workload_deviation', 'Reason for absence_1 Certain infectious and parasitic diseases', 'Reason for absence_10 Diseases of the respiratory system', 'Reason for absence_11 Diseases of

In [4]:
# Configurar el experimento de MLflow
mlflow.set_experiment("Absenteeism_Prediction_Experiments")

# Diccionario de modelos que vamos a probar
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}

print("Iniciando experimentación de modelos...")

for name, model in models.items():
    # Iniciar una nueva ejecución en MLflow para cada modelo
    with mlflow.start_run(run_name=name):
        print(f"\n--- Entrenando y evaluando: {name} ---")

        # 1. Registrar el nombre del modelo como un parámetro
        mlflow.log_param("model_name", name)

        # 2. Entrenar el modelo con los datos escalados
        model.fit(X_train_scaled, y_train)

        # 3. Realizar predicciones sobre el conjunto de prueba
        y_pred = model.predict(X_test_scaled)
        y_proba = model.predict_proba(X_test_scaled)[:, 1] # Probabilidades para ROC AUC

        # 4. Calcular las métricas de evaluación
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_proba)

        # 5. Registrar las métricas en MLflow para poder compararlas después
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)
        mlflow.log_metric("roc_auc", roc_auc)
        
        # 6. Registrar los hiperparámetros del modelo
        mlflow.log_params(model.get_params())

        # 7. Registrar el modelo entrenado como un "artefacto" en MLflow
        mlflow.sklearn.log_model(model, f"model_{name.replace(' ', '_')}")

        print(f"  Accuracy: {accuracy:.4f}")
        print(f"  F1-Score: {f1:.4f}")
        print(f"  ROC AUC: {roc_auc:.4f}")

print("\n✅ Experimentación de modelos finalizada.")

Iniciando experimentación de modelos...

--- Entrenando y evaluando: Logistic Regression ---


2025/10/12 12:00:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/12 12:01:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


  Accuracy: 0.9417
  F1-Score: 0.9250
  ROC AUC: 0.9902
🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/383079581240596013/runs/50947a200b5e4680a2f58ff9e7b9bf17
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/383079581240596013

--- Entrenando y evaluando: Decision Tree ---


2025/10/12 12:01:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/12 12:01:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


  Accuracy: 1.0000
  F1-Score: 1.0000
  ROC AUC: 1.0000
🏃 View run Decision Tree at: http://127.0.0.1:5000/#/experiments/383079581240596013/runs/1eb2d1b3fce64c92b93343662907654b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/383079581240596013

--- Entrenando y evaluando: Random Forest ---


2025/10/12 12:01:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/12 12:01:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


  Accuracy: 1.0000
  F1-Score: 1.0000
  ROC AUC: 1.0000
🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/383079581240596013/runs/7f1496c99e9242b18b1dfb7254d84b8e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/383079581240596013

--- Entrenando y evaluando: Gradient Boosting ---


2025/10/12 12:01:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/12 12:01:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


  Accuracy: 1.0000
  F1-Score: 1.0000
  ROC AUC: 1.0000
🏃 View run Gradient Boosting at: http://127.0.0.1:5000/#/experiments/383079581240596013/runs/74e256fe650d43c5b07993e3d968b563
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/383079581240596013

✅ Experimentación de modelos finalizada.


Los modelos basados en árboles (Decision Tree, Random Forest) son extremadamente buenos para encontrar "atajos" en los datos. 

Si hay una columna en tus datos que es básicamente la respuesta, el árbol la encontrará y creará una regla simple como: SI columna_tramposa > 0.5 ENTONCES la respuesta es 1. 

Por eso obtienen un 100% de precisión. La Regresión Logística, al ser un modelo más simple, es menos susceptible a estos atajos, por eso su resultado es más realista.

In [5]:
# --- Paso de Diagnóstico: Investigar Correlaciones ---
# Vamos a calcular la correlación de todas las columnas con nuestra variable objetivo.
# Una correlación muy cercana a 1 o -1 es una señal de fuga de datos.

print("Calculando correlaciones con la variable objetivo...")
correlations = df.corr()['Target_Binary'].sort_values(ascending=False)

print("\nTop 10 correlaciones más altas con 'Target_Binary':")
print(correlations.head(10))

print("\nTop 10 correlaciones más bajas (más negativas) con 'Target_Binary':")
print(correlations.tail(10))

# Identificar columnas con correlación perfecta (¡las culpables!)
leaky_features = correlations[abs(correlations) > 0.99].index.tolist()

# El Target_Binary siempre tendrá una correlación de 1.0 consigo mismo, así que lo removemos de la lista.
if 'Target_Binary' in leaky_features:
    leaky_features.remove('Target_Binary')

if leaky_features:
    print(f"\n🚨 ¡Fuga de datos detectada! La(s) siguiente(s) característica(s) son probablemente la causa: {leaky_features}")
else:
    print("\n✅ No se detectó una fuga de datos obvia por correlación. El overfitting podría deberse a la complejidad del modelo.")

Calculando correlaciones con la variable objetivo...

Top 10 correlaciones más altas con 'Target_Binary':
Target_Binary                                    1.000000
Absenteeism time in hours                        0.818868
Reason for absence_26 Unjustified absence        0.296590
Son                                              0.285075
Lifestyle_risk_score                             0.238655
Service_group_Long                               0.235744
Age_group_Middle-aged                            0.232805
Transportation expense                           0.223499
Reason for absence_22 Patient follow-up (CID)    0.210285
Distance_group_Moderate                          0.202714
Name: Target_Binary, dtype: float64

Top 10 correlaciones más bajas (más negativas) con 'Target_Binary':
Month of absence_January                     -0.130176
ID                                           -0.134605
Month of absence_February                    -0.135235
Penalty_risk_score                          

Para validar creamos un diagnostico de correlacines para ver si hay data leakage pero vemos que hay en esos modelos overfitting por la naturaleza del modelo

Ahora regularemos los modelos

In [6]:
# Configurar el experimento de MLflow
mlflow.set_experiment("Absenteeism_Prediction_Experiments_Regularized") # Nuevo nombre de experimento

# Diccionario de modelos con versiones regularizadas para combatir el overfitting
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Decision Tree (Regularized)': DecisionTreeClassifier(
        random_state=42,
        max_depth=5,            # Límite de profundidad
        min_samples_leaf=10     # Mínimo de muestras por hoja
    ),
    'Random Forest (Regularized)': RandomForestClassifier(
        random_state=42,
        max_depth=8,            # Límite de profundidad para cada árbol
        min_samples_leaf=5,     # Mínimo de muestras por hoja
        n_estimators=150        # Número de árboles
    ),
    'Gradient Boosting (Regularized)': GradientBoostingClassifier(
        random_state=42,
        max_depth=4,            # Límite de profundidad para cada árbol
        n_estimators=100
    )
}

print("Iniciando experimentación con modelos regularizados...")

for name, model in models.items():
    with mlflow.start_run(run_name=name):
        print(f"\n--- Entrenando y evaluando: {name} ---")

        mlflow.log_param("model_name", name)
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_proba = model.predict_proba(X_test_scaled)[:, 1]

        # Calcular y registrar métricas
        metrics = {
            "accuracy": accuracy_score(y_test, y_pred),
            "precision": precision_score(y_test, y_pred),
            "recall": recall_score(y_test, y_pred),
            "f1_score": f1_score(y_test, y_pred),
            "roc_auc": roc_auc_score(y_test, y_proba)
        }
        mlflow.log_metrics(metrics)
        mlflow.log_params(model.get_params())
        mlflow.sklearn.log_model(model, f"model_{name.replace(' ', '_')}")

        print(f"  Accuracy: {metrics['accuracy']:.4f}")
        print(f"  F1-Score: {metrics['f1_score']:.4f}")
        print(f"  ROC AUC: {metrics['roc_auc']:.4f}")

print("\n✅ Experimentación finalizada.")

2025/10/12 12:14:41 INFO mlflow.tracking.fluent: Experiment with name 'Absenteeism_Prediction_Experiments_Regularized' does not exist. Creating a new experiment.


Iniciando experimentación con modelos regularizados...

--- Entrenando y evaluando: Logistic Regression ---


2025/10/12 12:14:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/12 12:14:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


  Accuracy: 0.9417
  F1-Score: 0.9250
  ROC AUC: 0.9902
🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/541240116825496883/runs/09e685330e56412b97a07d02bae6621a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/541240116825496883

--- Entrenando y evaluando: Decision Tree (Regularized) ---


2025/10/12 12:14:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/12 12:15:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


  Accuracy: 1.0000
  F1-Score: 1.0000
  ROC AUC: 1.0000
🏃 View run Decision Tree (Regularized) at: http://127.0.0.1:5000/#/experiments/541240116825496883/runs/de18a03c069c49b78cfa63b55ec3afee
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/541240116825496883

--- Entrenando y evaluando: Random Forest (Regularized) ---


2025/10/12 12:15:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/12 12:15:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


  Accuracy: 1.0000
  F1-Score: 1.0000
  ROC AUC: 1.0000
🏃 View run Random Forest (Regularized) at: http://127.0.0.1:5000/#/experiments/541240116825496883/runs/13f5798431dd4649a47800e1713533a7
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/541240116825496883

--- Entrenando y evaluando: Gradient Boosting (Regularized) ---


2025/10/12 12:15:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/12 12:15:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


  Accuracy: 1.0000
  F1-Score: 1.0000
  ROC AUC: 1.0000
🏃 View run Gradient Boosting (Regularized) at: http://127.0.0.1:5000/#/experiments/541240116825496883/runs/d72e6e3927184d02b141bfb15d405434
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/541240116825496883

✅ Experimentación finalizada.
